In [1]:
import os
import yaml
import numpy as np

In [2]:
import textwrap

In [3]:
base_dir = '/glade/work/ksha/DWC_runs/CONUS_GP_clean/'

for i_mem in range(10):
    config_name = os.path.realpath(base_dir+'model_clim_CESM_base.yml')
    with open(config_name, 'r') as stream:
        conf = yaml.safe_load(stream)
    
    filepath_ref = conf['data']['boundary']['save_loc']
    
    filepath_new = filepath_ref[:-8] + str(i_mem) + filepath_ref[-7:]
    
    conf['data']['boundary']['save_loc'] = filepath_new 
    conf['data']['boundary']['save_loc_surface'] = filepath_new
    
    savename_ref =  conf['predict']['save_forecast']
    savename_new = savename_ref[:-3] + '{:02d}'.format(i_mem) + '/'
    conf['predict']['save_forecast'] = savename_new
    
    with open(base_dir+'model_clim_CESM_mem{:02d}.yml'.format(i_mem), 'w') as out:
        yaml.safe_dump(conf, out, default_flow_style=False)
    

In [6]:
filepath_new

'/glade/derecho/scratch/ksha/DWC_data/CONUS_domain_GP/dscale_CESM/CESM_GP_mem9_*.zarr'

In [4]:
for i_mem in range(10):
    # build the output filename
    script_fname = os.path.join(
        base_dir,
        f'launch_clim_CESM_mem{i_mem:02d}.sh'
    )
    
    # build the script contents, with no extra indentation
    body = textwrap.dedent(f"""\
        #!/bin/bash
        #PBS -A P48500028
        #PBS -N Pred_mem{i_mem:02d}
        #PBS -l walltime=12:00:00
        #PBS -l select=1:ncpus=64:ngpus=1
        #PBS -q main
        #PBS -j oe
        #PBS -k eod
        #PBS -r n
        # Load modules
        module purge
        module load ncarenv/24.12
        module reset
        module load gcc craype cray-mpich cuda cudnn conda
        conda activate credit-new
        # Export environment variables
        export LSCRATCH=/glade/derecho/scratch/ksha/
        export LOGLEVEL=INFO
        export NCCL_DEBUG=INFO
        export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7,8
        export NCCL_SOCKET_IFNAME=hsn
        export MPICH_GPU_MANAGED_MEMORY_SUPPORT_ENABLED=1
        export MPICH_OFI_NIC_POLICY=GPU
        export MPICH_GPU_SUPPORT_ENABLED=1
        export NCCL_IB_DISABLE=1
        export NCCL_CROSS_NIC=1
        export NCCL_NCHANNELS_PER_NET_PEER=4
        export MPICH_RDMA_ENABLED_CUDA=1
        export NCCL_NET="AWS Libfabric"
        export NCCL_NET_GDR_LEVEL=PBH
        export FI_CXI_DISABLE_HOST_REGISTER=1
        export FI_CXI_OPTIMIZED_MRS=false
        export FI_MR_CACHE_MONITOR=userfaultfd
        export FI_CXI_DEFAULT_CQ_SIZE=131072
        # Launch MPIs
        nodes=( $( cat $PBS_NODEFILE ) )
        echo nodes: $nodes
        # Find headnode's IP:
        head_node=${{nodes[0]}}
        head_node_ip=$(host $head_node | awk '{{print $NF}}')
        MASTER_ADDR=$head_node_ip MASTER_PORT=1234 mpiexec -n 1 --ppn 4 --cpu-bind none \\
            python /glade/u/home/ksha/credit-mini/applications/WRF_pred_future.py \\
            -c model_clim_CESM_mem{i_mem:02d}.yml
        """)
    
    # write out the script
    with open(script_fname, 'w') as f:
        f.write(body)
        

In [5]:
f = open(base_dir+'batch_CESM.sh', 'w')
for i_mem in range(10):
    print(f'qsub launch_clim_CESM_mem{i_mem:02d}.sh', file=f)
f.close()